<a href="https://colab.research.google.com/github/vicotrbb/machine_learning/blob/master/projects/pneumonia_detection/poc_pneumonia_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementação de um classificar de CV para detecção de pneumonia

In [6]:
import numpy as np
import os
import cv2
import random
import pickle
from zipfile import ZipFile

import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

In [4]:
!ls 'drive/My Drive/train'

NORMAL	PNEUMONIA


In [9]:
def unzip_data(datadir):
  with ZipFile(datadir, 'r') as zip_ref:
    zip_ref.extractall(datadir.replace('zip', ''))


In [15]:
def prepare_data(datadir, img_size=28):
	file_list = []
	class_list = []
	x = []
	y = []
	data = []
	error = False
	categories = ['NORMAL', 'PNEUMONIA']

	for category in categories: 
		path = os.path.join(datadir, category)
		class_index = categories.index(category)
		for img in os.listdir(path):
			try:
				img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
				new_array = cv2.resize(img_array, (img_size, img_size))
				data.append([new_array, class_index])
			except Exception as e:
				error = True
				pass

	random.shuffle(data)

	for features, label in data:
		x.append(features)
		y.append(label)

	x = np.array(x).reshape(-1, img_size, img_size, 1)
	if error:
		print('Erro ao processar algums imagens')
	else:
		print('Imagens processadas com sucesso')
	return x, y


In [16]:
class Pneumonia:

	def __init__(self, train_dir, test_dir):
		self.model = Sequential()
		self.img_size = 28
		self.categories = ['NORMAL', 'PNEUMONIA']
		self.x_train, self.y_train = prepare_data(train_dir)
		self.x_test, self.y_test = prepare_data(test_dir)


	def create_train_model(self):
		self.model = Sequential()
		self.x_train /= 255

		# convolutional layers
		self.model.add(Conv2D(32, (3, 3), input_shape = x.shape[1:]))
		self.model.add(Activation("relu"))
		self.model.add(MaxPooling2D(pool_size=(2,2)))

		self.model.add(Conv2D(64, (3, 3)))
		self.model.add(Activation("relu"))
		self.model.add(MaxPooling2D(pool_size=(2,2)))

		self.model.add(Conv2D(64, (3, 3)))
		self.model.add(Activation("relu"))
		self.model.add(MaxPooling2D(pool_size=(2,2)))
		self.model.add(Dropout(0.25))

		# hidden layers
		self.model.add(Flatten())
		self.model.add(Dense(128))
		self.model.add(Activation("relu"))

		self.model.add(Dense(128))
		self.model.add(Activation("relu"))

		# output layer
		self.model.add(Dense(2))
		self.model.add(Activation("softmax"))

		# Compile
		self.model.compile(loss="sparse_categorical_crossentropy",
				optimizer="adam",
				metrics=["accuracy"])

		self.model.fit(self.x_train, self.y_test, batch_size=32, epochs=40, validation_split=0.1,
			verbose=1, validation_data=(self.x_test, self.y_test))

		return self


	def predict_image(self, file):
		img_array = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
		new_array = cv2.resize(img_array, (self.img_size, self.img_size))
		img = new_array.reshape(-1, self.img_size, self.img_size, 1)
		prediction = model.predict([img])
		prediction = list(prediction[0])
		print(self.categories[prediction.index(max(prediction))])


In [ ]:
model = Pneumonia('drive/My Drive/train', 'drive/My Drive/chest_xray_lite_dataset/test')

In [87]:
x_train = model.x_train
y_train = model.y_train
x_test = model.x_test
y_test = model.y_test

y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

In [88]:
x_train = x_train / 255

In [89]:
print(x_train.shape, x_train.dtype)

(5214, 28, 28, 1) float64


In [90]:
print(y_train.shape, y_train.dtype)

(5214, 1) int64


In [92]:
ml_model = Sequential()

# convolutional layers
ml_model.add(Conv2D(32, (3, 3), input_shape = x_train.shape[1:]))
ml_model.add(Activation("relu"))
ml_model.add(MaxPooling2D(pool_size=(2,2)))

ml_model.add(Conv2D(64, (3, 3)))
ml_model.add(Activation("relu"))
ml_model.add(MaxPooling2D(pool_size=(2,2)))

ml_model.add(Conv2D(64, (3, 3)))
ml_model.add(Activation("relu"))
ml_model.add(MaxPooling2D(pool_size=(2,2)))
ml_model.add(Dropout(0.25))

# hidden layers
ml_model.add(Flatten())
ml_model.add(Dense(128))
ml_model.add(Activation("relu"))

ml_model.add(Dense(128))
ml_model.add(Activation("relu"))

# output layer
ml_model.add(Dense(2))
ml_model.add(Activation("softmax"))

# Compile
ml_model.compile(loss="sparse_categorical_crossentropy",
      optimizer="adam",
      metrics=["accuracy"])

ml_model.fit(x_train, y_train, batch_size=128, epochs=100, verbose=1)

Epoch 1/100
41/41 [==============================] - 4s 103ms/step - loss: 0.5547 - accuracy: 0.7440
Epoch 2/100
41/41 [==============================] - 4s 103ms/step - loss: 0.3646 - accuracy: 0.8274
Epoch 3/100
41/41 [==============================] - 4s 103ms/step - loss: 0.2860 - accuracy: 0.8744
Epoch 4/100
41/41 [==============================] - 4s 104ms/step - loss: 0.2461 - accuracy: 0.8949
Epoch 5/100
41/41 [==============================] - 4s 104ms/step - loss: 0.2403 - accuracy: 0.9010
Epoch 6/100
41/41 [==============================] - 4s 105ms/step - loss: 0.1836 - accuracy: 0.9298
Epoch 7/100
41/41 [==============================] - 4s 106ms/step - loss: 0.1599 - accuracy: 0.9373
Epoch 8/100
41/41 [==============================] - 4s 104ms/step - loss: 0.1394 - accuracy: 0.9467
Epoch 9/100
41/41 [==============================] - 4s 102ms/step - loss: 0.1380 - accuracy: 0.9517
Epoch 10/100
41/41 [==============================] - 4s 103ms/step - loss: 0.1215 - accura

In [93]:
ml_model.save('pneumonia_model.hdf5')

In [96]:
categories = ['NORMAL', 'PNEUMONIA']

In [95]:
def predict_image(file):
  img_array = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
  new_array = cv2.resize(img_array, (28, 28))
  img = new_array.reshape(-1, 28, 28, 1)
  prediction = ml_model.predict([img])
  prediction = list(prediction[0])
  print(categories[prediction.index(max(prediction))])

In [100]:
!ls 'drive/My Drive/chest_xray_lite_dataset/val'

NORMAL	PNEUMONIA


In [99]:
predict_image('drive/My Drive/chest_xray_lite_dataset/test/NORMAL/NORMAL2-IM-0339-0001.jpeg')

NORMAL


In [101]:
x_val, y_val = prepare_data('drive/My Drive/chest_xray_lite_dataset/val')

Imagens processadas com sucesso


In [105]:
y_val = np.array(y_val).reshape(-1, 1)

In [110]:
scores = ml_model.evaluate(x_val, y_val)
print("%.2f%% (+/- %.2f%%)" % (np.mean(scores[1] * 100), np.std(scores[1] * 100)))

1/1 [==============================] - 0s 2ms/step - loss: 23.0543 - accuracy: 0.8125
81.25% (+/- 0.00%)
